In [1]:
! pip install fosforml

     |████████████████████████████████| 51kB 490kB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 42.1MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 34.4MB/s eta 0:00:01
     |████████████████████████████████| 18.3MB 40.2MB/s eta 0:00:01
     |████████████████████████████████| 307kB 41.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 133kB 38.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 13.5MB/s eta 0:00:01
     |████████████████████████████████| 98.2MB 133kB/s  eta 0:00:01     |████████████▋                   | 38.9MB 39.0MB/s eta 0:00:02
     |████████████████████████████████| 13.1MB 33.5MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 53.6MB/s eta 0:00:01
     |████████████████████████████████| 174kB 41.5MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 33.4MB/s eta 0:

     |████████████████████████████████| 8.3MB 49.7MB/s eta 0:00:01
     |████████████████████████████████| 16.4MB 35.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 9.0MB/s  eta 0:00:01
     |████████████████████████████████| 235kB 47.7MB/s eta 0:00:01
     |████████████████████████████████| 348kB 44.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 37.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 38.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 12.3MB/s eta 0:00:01
     |████████████████████████████████| 143kB 36.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 50.4MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 48.9MB/s eta 0:00:01
     |████████████████████████████████| 450kB 38.3MB/s eta 0:00:01
     |████████████████████████████████| 163kB 43.1MB/s eta 0:00:0

     |████████████████████████████████| 4.4MB 54.8MB/s eta 0:00:01
     |████████████████████████████████| 307kB 27.2MB/s eta 0:00:01
     |████████████████████████████████| 4.6MB 35.1MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 35.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 46.4MB/s eta 0:00:01
     |████████████████████████████████| 122kB 43.0MB/s eta 0:00:01
ERROR: refractml 1.0.3 has requirement cloudpickle==1.6.0, but you'll have cloudpickle 2.2.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement urllib3==1.26.15, but you'll have urllib3 1.26.19 which is incompatible.
ERROR: refractio 2.1.5.2 has requirement pandas==2.0.0, but you'll have pandas 2.2.2 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3.8", but you'll have scikit-learn 1.3.2 which is incompatible.
ERROR: mlflo

In [ ]:
from snowflake.snowpark.session import Session
from snowflake.ml.registry.registry import Registry
import datetime, json
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

def get_feature_columns(session, input_table, target_column):

    df = session.table(input_table)
    # schema_fields = df.schema.fields
    features = df.columns
    features.remove(target_column)
    desc_sql_query = 'DESCRIBE TABLE '+ input_table
    df_schema = session.sql(desc_sql_query).collect()
    categorical_types = ['VARCHAR', 'CHAR', 'STRING', 'TEXT', 'BOOL']
    categorical_features = []
    for row in df_schema:
        for typ in categorical_types:
            if typ in row['type']:
                categorical_features.append(row['name'])
                break
    numerical_features = list(set(features) - set(categorical_features))
    print("numerical features: ", numerical_features)
    # identify columns for labelencoding and onehotencoding
    le_column_features = categorical_features
    oh_column_features = []
    if len(categorical_features) >= 1:
        for column in categorical_features:
            if df.select(df[column]).distinct().count() < 10:
                oh_column_features.append(column)
    print("le_features: ", le_column_features)
    print("oh_features: ", oh_column_features)
    return df, numerical_features, le_column_features, oh_column_features


def create_and_run_preprocessing(df, numerical_features, le_column_features, oh_column_features):
    # pipeline steps
    print(df.show(n=10))
    categorical_pp = numerical_pp = dict()
    for column in le_column_features:
        key = 'le_'+column
        categorical_pp[key] = LabelEncoder(input_cols=column, output_cols=column)
    if len(oh_column_features) > 0:
        categorical_pp['oh_enc'] = OneHotEncoder(input_cols=oh_column_features, output_cols=oh_column_features,
                                                 handle_unknown='ignore')
    numerical_pp['scaler'] = MinMaxScaler(input_cols=numerical_features, output_cols=numerical_features)

    steps = [(key, categorical_pp[key]) for key in categorical_pp if categorical_pp[key] != []] + \
            [(key, numerical_pp[key]) for key in numerical_pp if numerical_features != []]

    print("df.columns =", df.columns)
    print("steps =", steps)
    # Run preprocessing pipeline steps
    df = Pipeline(steps=steps).fit(df).transform(df)
    # categorical_pp = {f'le_{column}':LabelEncoder(input_cols=column, output_cols=column) for column in le_column_features}
    # if len(oh_column_features)>0:
    #     categorical_pp['oh_enc'] = OneHotEncoder(input_cols=oh_column_features, output_cols=oh_column_features, handle_unknown='ignore')
    # numerical_pp = {
    #     'scaler': MinMaxScaler(input_cols=numerical_features, output_cols=numerical_features)
    # }
    # steps = [(key, categorical_pp[key]) for key in categorical_pp if categorical_pp[key]!=[]] + \
    # [(key, numerical_pp[key]) for key in numerical_pp if numerical_features!=[]]  
    # print("steps =", steps)   
    # Run preprocessing pipeline steps
    # df = Pipeline(steps=steps).fit(df).transform(df)
    # df_train, df_test = df.random_split(weights=[0.8, 0.2], seed=0)
    for col_name in df.schema.names:
        new_col = col_name.replace('.', '_')
        df = df.withColumnRenamed(col_name, new_col)
    print(df.show(n=10))
    return df


def batch_prediction(session, model_id, version_id, input_table, filter_cond='', output_table=''):
    reg = Registry(session=session)
    m = reg.get_model(model_id)
    mv = m.version(version_id)
    source = mv.get_metric('source')
    if source.upper() == 'EXPERIMENT':
        ds_query = "select METADATA:metrics:dataset_details as dataset from INFORMATION_SCHEMA.MODEL_VERSIONS where MODEL_NAME='"+model_id+"';"
        dataset_info = session.sql(ds_query).collect()
        dataset_list = dataset_info[0]['DATASET']
        target_column = eval(dataset_list)[0].get('target_column')
        df, numerical_features, le_column_features, oh_column_features = get_feature_columns(session, input_table, target_column)
        data = create_and_run_preprocessing(df, numerical_features, le_column_features, oh_column_features)
        remote_prediction = mv.run(data, function_name="predict")
        table_prefix = "PREDICTION_"
        new_table_name = (table_prefix + output_table).upper()
        session.write_pandas(remote_prediction, new_table_name, auto_create_table=True, overwrite=True)
        return output_table

    else:
        fetch_data_sql = "select * from " + input_table + " " + filter_cond
        df = session.sql(fetch_data_sql).collect()
        remote_prediction = mv.run(df, function_name="predict")
        table_prefix = "PREDICTION_"
        new_table_name = (table_prefix + output_table).upper()
        session.write_pandas(remote_prediction, new_table_name, auto_create_table=True, overwrite=True)
        return output_table


import time
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": "Enlightme#2024",
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC", }

start_time = time.time()
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

batch_prediction(session, 'MODEL_68BCE134_6D7A_4E24_9591_86266438ACF9_FDC_TESTBYPRAKHAR', 'V1', 'EMPLOYEE', '', '99999999')